## 读取人脸关键点，做出初始推荐

In [37]:
import dlib
import cv2
import numpy as np

# 设置正确的形状预测器文件路径
predictor_path = r"C:\Users\34568\Desktop\GlassesGAN\GlassesGAN_release\encoder4editing\shape_predictor_68_face_landmarks.dat"

# 加载人脸检测器和形状预测器
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# 加载图像
image_path = r"input/03.jpg"  # 替换为你的图片路径
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 检测人脸
faces = detector(gray)

# 提取量化参数
for face in faces:
    # 获取面部关键点
    landmarks = predictor(gray, face)
    
    # 眼睛中心点
    left_eye_center = (landmarks.part(36).x + landmarks.part(39).x) / 2, (landmarks.part(36).y + landmarks.part(39).y) / 2
    right_eye_center = (landmarks.part(42).x + landmarks.part(45).x) / 2, (landmarks.part(42).y + landmarks.part(45).y) / 2
    
    # 眼睛间距
    eye_distance = np.linalg.norm(np.array(left_eye_center) - np.array(right_eye_center))  # 眼睛间距
    
    # 脸宽
    left_cheek = (landmarks.part(0).x, landmarks.part(0).y)
    right_cheek = (landmarks.part(16).x, landmarks.part(16).y)
    face_width = np.linalg.norm(np.array(left_cheek) - np.array(right_cheek))  # 脸宽
    
    # 眼睛高度（垂直方向）
    eye_height = np.abs(landmarks.part(37).y - landmarks.part(41).y)  # 左眼上下高度差
    
    # 鼻梁宽度和高度
    nose_width = np.linalg.norm(np.array((landmarks.part(31).x, landmarks.part(31).y)) - np.array((landmarks.part(35).x, landmarks.part(35).y)))  # 鼻梁宽度
    nose_height = np.abs(landmarks.part(27).y - landmarks.part(33).y)  # 鼻梁高度
    
    # 下巴高度
    chin_height = np.abs(landmarks.part(8).y - landmarks.part(33).y)  # 下巴高度（从下巴到鼻尖）
    
    # 面部高度
    face_height = np.abs(landmarks.part(27).y - landmarks.part(8).y)  # 从鼻梁到下巴的高度
    
    # 计算比值
    eye_distance_ratio = eye_distance / face_width  # 眼睛间距与脸宽比值
    eye_height_ratio = eye_height / eye_distance  # 眼睛上下高度与眼睛间距比值
    nose_width_ratio = nose_width / face_width  # 鼻梁宽度与脸宽比值
    face_ratio = face_height / face_width  # 脸型的宽高比
    chin_to_nose_ratio = chin_height / nose_height  # 下巴高度与鼻梁高度比值
    
    # 输出量化参数
    print(f"眼睛间距: {eye_distance:.2f} 像素, 眼睛间距与脸宽比值: {eye_distance_ratio:.2f}")
    print(f"眼睛高度: {eye_height:.2f} 像素, 眼睛高度与眼睛间距比值: {eye_height_ratio:.2f}")
    print(f"鼻梁宽度: {nose_width:.2f} 像素, 鼻梁宽度与脸宽比值: {nose_width_ratio:.2f}")
    print(f"下巴高度: {chin_height:.2f} 像素, 下巴高度与鼻梁高度比值: {chin_to_nose_ratio:.2f}")
    print(f"面部高度: {face_height:.2f} 像素, 面部宽高比: {face_ratio:.2f}")

眼睛间距: 37.50 像素, 眼睛间距与脸宽比值: 0.42
眼睛高度: 6.00 像素, 眼睛高度与眼睛间距比值: 0.16
鼻梁宽度: 17.00 像素, 鼻梁宽度与脸宽比值: 0.19
下巴高度: 44.00 像素, 下巴高度与鼻梁高度比值: 1.47
面部高度: 74.00 像素, 面部宽高比: 0.83


In [38]:
import pandas as pd
from joblib import load

# 1️⃣ 加载模型
model = load('decision_tree_model.joblib')

# 2️⃣ 输入参数（使用字典方式更清晰）
new_data_dict = {
    'eye_distance_ratio': eye_distance_ratio,
    'eye_height_ratio': eye_height_ratio,
    'nose_width_ratio': nose_width_ratio,
    'face_ratio': face_ratio,
    'chin_to_nose_ratio': chin_to_nose_ratio
}

# 3️⃣ 将输入数据转换为 DataFrame（确保列顺序与训练时一致）
X_new = pd.DataFrame([new_data_dict])

# 4️⃣ 进行预测
predicted_class = model.predict(X_new)[0]

# 5️⃣ 输出最终结果
print("预测结果:", predicted_class)

预测结果: classic_square


In [40]:
# 风格名称列表
glass_style_names = [
    'classic_square',
    'retro_round',
    'cat_eye',
    'modern_large',
    'sporty',
    'minimalist',
    'vintage_metal',
    'elegant_small_round',
    'tech_savvy',
    'high_end',
    'trendy_youth',
    'unique_shape'
]

# 对应的特征向量（每个元素是一个6维元组）
glass_styles = [
    (5.0, -8.0, 10.0, 0.0, -5.0, 2.0),   # classic_square
    (6.5, -7.5, 2.0, 10.0, 2.0, 3.5),   # retro_round
    (7.0, -5.0, 8.0, 2.0, 12.0, -1.5),  # cat_eye
    (8.0, -9.0, 15.0, -2.0, -3.0, 4.0), # modern_large
    (6.0, -6.0, 12.0, -3.0, -2.0, 1.0), # sporty
    (4.0, -4.0, 4.0, 0.0, -4.0, 0.0),   # minimalist
    (5.5, -8.0, 6.0, 5.0, -6.0, -2.0),  # vintage_metal
    (3.5, -7.0, 1.0, 8.0, 5.0, 1.5),    # elegant_small_round
    (7.5, -5.5, 13.0, -1.0, -8.0, 3.0), # tech_savvy
    (8.5, -6.5, 10.0, 1.5, -4.5, 6.0),  # high_end
    (6.0, -3.0, 5.0, 6.0, 10.0, 0.0),   # trendy_youth
    (4.5, -9.5, 7.0, -5.0, -10.0, 2.5)  # unique_shape
]
# 创建字典：风格名 -> 向量
style_to_vector = dict(zip(style_names, glass_styles))
def get_vector_by_style(style_name):
    return style_to_vector.get(style_name, None)

vector = get_vector_by_style(predicted_class)

## 图片初始化和生成

In [41]:
import os
import numpy as np
from PIL import Image
import json

# 设置基础路径和环境变量
base_working_dir = r'C:\Users\34568\Desktop\GlassesGAN'
repo_name_dir = 'GlassesGAN_release'
msvc_path = r"C:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools\VC\Tools\MSVC\14.29.30133\bin\Hostx64\x64"
os.environ["PATH"] = msvc_path + ";" + os.environ.get("PATH", "")

# 输出目录
output_dir = f'{base_working_dir}/generated_images'
os.makedirs(output_dir, exist_ok=True)

# 切换到项目目录并导入核心类
os.chdir(f'{base_working_dir}/{repo_name_dir}')
from glasses_vton_inference import glasses_vton_inference

# 初始化参数
glasses_option = 'RG'
chosen_deeplab_epoch = 19
fitted_pca_fp = f'{base_working_dir}/fitted_pca_celebhq_dataset_results.joblib'
ave_add_glasses_diff_fp = f'{base_working_dir}/aveglassesdiff_celebhq_dataset_results_RG.npy'
resume_model_ckpt = f'{base_working_dir}/deeplab_epoch_{chosen_deeplab_epoch}.pth'

# 初始化 GlassesVton 类实例
gvton = glasses_vton_inference(
    fitted_pca_fp=fitted_pca_fp,
    ave_add_glasses_diff_fp=ave_add_glasses_diff_fp,
    base_working_dir=base_working_dir,
    temp_save_folder=f'{base_working_dir}/tempfolder',
    resume_model_ckpt=resume_model_ckpt,
    deeplab_script_location=f'{base_working_dir}/GlassesGAN_release/datasetGAN_release/datasetGAN',
    chosen_deeplab_epoch=chosen_deeplab_epoch,
    load_loc=base_working_dir,
    use_full_glasses_or_frames_mask='frames',
    run_tests=False,
    outer_dilation_factor=5,
    outer_blur_factor=12,
    inner_blur_factor=5,
    ideal_avg_glasses_frame_area=0.020,
    auto_clean=True
)
input_image, result_image, base_latent = gvton.embed_input_image(image_path, show_plots=False)
start_latent, edit_image, _ = gvton.add_avg_glasses(input_image, base_latent, base_bias=1, show_plots=False, auto_pick_bias=True, return_blended_image=True)

Loading e4e over the pSp framework from checkpoint: pretrained_models/e4e_ffhq_encode.pt
Model successfully loaded!
Model checkpoint: C:\Users\34568\Desktop\GlassesGAN/deeplab_epoch_19.pth (valid)
DeepLab script: C:\Users\34568\Desktop\GlassesGAN/GlassesGAN_release/datasetGAN_release/datasetGAN\test_deeplab_cross_validation.py (valid)

Removing temporary folder

Copying model to temporary working directory
Aligned image has shape: (1024, 1024)
Inference took 0.0915 seconds.

Saving image and fake groundtruth maps to temporary file location


0it [00:00, ?it/s]

Resizing image to (512,512,3)


1it [00:00,  1.91it/s]

Resizing image to (512,512,3)


...


Running deeplab
Opt {'exp_dir': 'C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/model', 'batch_size': 64, 'category': 'face', 'debug': False, 'dim': [512, 512, 5088], 'deeplab_res': 512, 'number_class': 4, 'testing_data_number_class': 4, 'max_training': 7, 'stylegan_ver': '1', 'annotation_data_from_w': False, 'annotation_mask_path': './custom_data/annotation/training_data', 'testing_path': 'C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/input_images', 'average_latent': './custom_data/training_latent/avg_latent_stylegan1.npy', 'annotation_image_latent_path': './custom_data/training_latent/latent_stylegan1.npy', 'stylegan_checkpoint': './checkpoints/stylegan_pretrain/karras2019stylegan-ffhq-1024x1024_old_serialization.pt', 'model_num': 10, 'upsample_mode': 'bilinear'}
args Namespace(exp='C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/face_34.json', resume='C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/model', cross_validate=False, validation_number=0, chosen_deeplab_epoch=19)
R

0it [00:00, ?it/s]

Resizing image to (512,512,3)


1it [00:00,  2.05it/s]



Running deeplab
Opt {'exp_dir': 'C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/model', 'batch_size': 64, 'category': 'face', 'debug': False, 'dim': [512, 512, 5088], 'deeplab_res': 512, 'number_class': 4, 'testing_data_number_class': 4, 'max_training': 7, 'stylegan_ver': '1', 'annotation_data_from_w': False, 'annotation_mask_path': './custom_data/annotation/training_data', 'testing_path': 'C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/input_images', 'average_latent': './custom_data/training_latent/avg_latent_stylegan1.npy', 'annotation_image_latent_path': './custom_data/training_latent/latent_stylegan1.npy', 'stylegan_checkpoint': './checkpoints/stylegan_pretrain/karras2019stylegan-ffhq-1024x1024_old_serialization.pt', 'model_num': 10, 'upsample_mode': 'bilinear'}
args Namespace(exp='C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/face_34.json', resume='C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/model', cross_validate=False, validation_number=0, chosen_deeplab_epoch=19)
R

In [42]:
def plot_single_image(image, title='Image', figsize=(4, 4), dpi=150, save_path_name=None):
    plt.close('all')
    
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    if isinstance(image, str):
        img = plt.imread(image)
    else:
        img = image
    
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off') 

    if save_path_name:
        plt.savefig(save_path_name, bbox_inches='tight', dpi=dpi)
    else:
        plt.show()

## 多臂老虎机

In [43]:
from sklearn.metrics.pairwise import cosine_similarity
# 获取该款式的索引
initial_style_idx = glass_style_names.index(predicted_class)
def get_initial_scores(glass_styles, initial_style_idx, method='cosine'):
    glass_array = np.array(glass_styles)
    initial_vec = glass_array[initial_style_idx].reshape(1, -1)
    
    if method == 'cosine':
        scores = cosine_similarity(glass_array, initial_vec).flatten()
    elif method == 'euclidean':
        scores = -np.linalg.norm(glass_array - initial_vec, axis=1)
    else:
        raise ValueError("method must be 'cosine' or 'euclidean'")
    
    # 可选：归一化得分到 [0,1] 区间
    scores = (scores - scores.min()) / (scores.max() - scores.min())
    
    return scores
# 计算12款眼镜的初始得分
scores = get_initial_scores(glass_styles, initial_style_idx, method='cosine')

# 打印每款眼镜和对应得分
for name, score in zip(glass_style_names, scores):
    print(f"{name:<20} : {score:.4f}")

classic_square       : 1.0000
retro_round          : 0.3309
cat_eye              : 0.1506
modern_large         : 0.9548
sporty               : 0.9166
minimalist           : 0.9305
vintage_metal        : 0.8126
elegant_small_round  : 0.1092
tech_savvy           : 0.9474
high_end             : 0.9177
trendy_youth         : 0.0000
unique_shape         : 0.8494


In [44]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# 输出容器
output = widgets.Output()

class MultiArmBanditWithFeedback:
    def __init__(self, glass_styles, glass_style_names, initial_scores=None):
        self.glass_styles = glass_styles
        self.glass_style_names = glass_style_names
        self.num_arms = len(glass_styles)
        self.rewards = np.zeros(self.num_arms)
        self.num_trials = np.zeros(self.num_arms)
        self.epsilon = 0.2
        
        # 存储初始得分
        self.initial_scores = initial_scores
        if initial_scores is not None:
            self.initial_scores = np.array(initial_scores)
        
    def select_arm(self):
        # 当试验次数较少时，同时考虑初始得分和当前奖励估计
        if np.sum(self.num_trials) < 5 and self.initial_scores is not None:
            # 结合初始得分和当前奖励估计
            current_estimates = self.rewards / (self.num_trials + 1e-5)
            combined_scores = current_estimates * 0.7 + self.initial_scores * 0.3  # 加权组合
            return np.argmax(combined_scores)
            
        # 否则使用标准的ε-贪心策略
        if np.random.rand() > self.epsilon:
            return np.argmax(self.rewards / (self.num_trials + 1e-5))
        else:
            return np.random.choice(range(self.num_arms))

    def update(self, arm_index, reward):
        self.num_trials[arm_index] += 1
        self.rewards[arm_index] += reward

    def generate_and_show_image(self, arm_index):
        vector = self.glass_styles[arm_index]
        Size, Height, Squareness, Round_Shrink, Cateye, Thicken = vector

        num_pcs = 6
        latent = start_latent.copy()
        for PC_num, PC_bias in enumerate((Size, Height, Squareness, Round_Shrink, Cateye, Thicken)):
            run_gen = True if PC_num == (num_pcs - 1) else False
            img, latent = gvton.e4e.run_gen_add_pc_direction_bias(
                start_latent=latent,
                fitted_pca=gvton.fitted_pca,
                bias=PC_bias,
                PC_num=PC_num,
                run_gen=run_gen
            )

        blends, _, _ = gvton.blend_in_edits(edits=[img], input_image=input_image)

        # 绘制新图像
        plt.figure(figsize=(4, 4), dpi=150)
        plot_single_image(image=blends[0],
                          title=f"Style: {self.glass_style_names[arm_index]}",
                          figsize=(4, 4),
                          dpi=150)

    def on_like_click(self, arm_index):
        print(f"👍 用户满意: {self.glass_style_names[arm_index]}")
        self.update(arm_index, reward=0.1)
        self.next_step()

    def on_dislike_click(self, arm_index):
        print(f"👎 用户不满意: {self.glass_style_names[arm_index]}")
        self.update(arm_index, reward=-0.1)
        self.next_step()

    def show_feedback_buttons(self, arm_index):
        like_btn = widgets.Button(description="满意")
        dislike_btn = widgets.Button(description="不满意")

        like_btn.on_click(lambda btn: self.on_like_click(arm_index))
        dislike_btn.on_click(lambda btn: self.on_dislike_click(arm_index))

        button_box = widgets.HBox([like_btn, dislike_btn])
        display(button_box)

    def next_step(self):
        # 先清除所有现有输出
        with output:
            clear_output(wait=True)
            arm_index = self.select_arm()
            self.generate_and_show_image(arm_index)
            self.show_feedback_buttons(arm_index)
            plt.show()  # 确保图像显示

    def run(self):
        print("开始推荐...")
        with output:
            clear_output(wait=True)  # 确保开始时输出区域为空
        self.next_step()

# 假设已经定义了plot_single_image函数
def plot_single_image(image, title='Image', figsize=(4, 4), dpi=150, save_path_name=None):
    plt.close('all')
    
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    if isinstance(image, str):
        img = plt.imread(image)
    else:
        img = image
    
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off') 

    if save_path_name:
        plt.savefig(save_path_name, bbox_inches='tight', dpi=dpi)
    else:
        plt.show()


# 先显示 output 容器
display(output)

# 启动推荐系统，传入初始得分
bandit = MultiArmBanditWithFeedback(glass_styles, glass_style_names, initial_scores=scores)
bandit.run()

Output()

开始推荐...
